In [1]:
import requests
from bs4 import BeautifulSoup
import json
gs25_url = "https://gs25.gsretail.com/gscvs/ko/store-services/locations#;"

In [18]:
payload= {"pageNum" : "1",
"pageSize" : "100000",
"searchShopName" : "",
"searchSido" : "",
"searchGugun" : "",
"searchDong" : "",
"searchType" : "",
"searchTypeService" : "0",
"searchTypeToto" : "0",
"searchTypeCafe25" : "0",
"searchTypeInstant" : "0",
"searchTypeDrug" : "0",
"searchTypeSelf25" : "0",
"searchTypePost" : "0",
"searchTypeATM" : "0",
"searchTypeWithdrawal" : "0",
"searchTypeTaxrefund" : "0",
"searchTypeSmartAtm" : "0",
"searchTypeSelfCookingUtensils" : "0",
"searchTypeDeliveryService" : "0",
"searchTypeParcelService" : "0",
"searchTypePotatoes" : "0",
"searchTypeCardiacDefi" : "0",
"searchTypeFishShapedBun" : "0",
"searchTypeWine25" : "0",
"searchTypeGoPizza" : "0",
"searchTypeSpiritWine" : "0",
"searchTypeFreshGanghw" : "0",
"searchTypeMusinsa" : "0",
"searchTypePosa" : "0",}
header = {
    'host' : 'gs25.gsretail.com',
    'origin' : 'https://gs25.gsretail.com',
    'referer' : 'https://gs25.gsretail.com/gscvs/ko/store-services/locations'
}

In [19]:
url = "https://gs25.gsretail.com/gscvs/ko/store-services/locationList?CSRFToken={}"
with requests.Session() as s:
    r = s.get(gs25_url)
    bs = BeautifulSoup(r.text)
    csrf = bs.find("form", id='CSRFForm').find("input")['value']
    r2 = s.post(url.format(csrf), data=payload, headers=header)

In [24]:
ress = json.loads(r2.json())['results']
len(ress)

17784

In [23]:
import pandas as pd
df = pd.DataFrame(ress)
df.iloc[:, 1:].duplicated().sum()

0

In [10]:
import pymysql
conn = pymysql.connect(host='127.0.0.1', user='play', passwd='123', database='sk17', port=3306)
cur = conn.cursor()

table_name = 'gs_stores_test'

insert_q = f"INSERT IGNORE INTO {table_name} VALUES (%s, %s, %s, %s, %s, %s)"
create_q = f"""CREATE TABLE {table_name} (
    shopCode varchar(10) not null,
    shopName varchar(20) default null,
    address varchar(100)  default null default null,
    offeringService varchar(60)  not null,
    longs varchar(20) default null,
    lat varchar(20) default null
)"""

In [11]:
#초기화
cur.execute('show tables')
tables = [row[0] for row in cur.fetchall()]
if table_name in tables:
    cur.execute(f'drop table {table_name}')
cur.execute(create_q)

0

In [12]:
service_code = {
    'toto' : '1',
    'cafe25' : '2',
    'post' : '3',
    'withdrawal' : '4',
    'parcel_service' : '5',
    'drug' : '6',
    'atm' : '7',
    'smart_atm' : '8',
    'musinsa' : '9',
    'potatoes' : '10',
    'fish_shaped_bun' : '11',
    'posa' : '12',
    'instant' : '13',
    'cardiac_defi' : '14',
    'spirit_wine' : '15',
    'delivery_service' : '16',
    'self_cook' : '17',
    'fresh_ganghw' : '18',
    'gopizza' : '19',
    'tax' : '20'
}

In [13]:
for res in ress:
    row = ['']*6
    if 'shopCode' in res:
        row[0] = res['shopCode']
    if 'shopName' in res:
        row[1] = res['shopName']
    if 'address' in res:
        row[2] = res['address']
    if 'offeringService' in res:
        row[3] = service_code[res['offeringService'][0]]
        for i in range(1, len(res['offeringService'])):
            row[3] = ', '.join((row[3], service_code[res['offeringService'][i]]))
    if 'longs' in res:
        row[4] = res['longs']
    if 'lat' in res:
        row[5] = res['lat']
    cur.execute(insert_q, row)

In [14]:
cur.execute(f'select * from {table_name}')

17784

In [16]:
conn.commit()